In [12]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Путь к файлу с облаком точек
# velodyne_file = '/home/dmitriiming/ResearchProject/Open3D-ML/examples/demo_data/KITTI/training/velodyne/000008.bin'
clustered_ply_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000008.bin'
# Загрузка данных из файла
with open(clustered_ply_file, 'rb') as f:
    # Чтение данных из файла в массив numpy
    velo_points = np.fromfile(f, dtype=np.float32).reshape(-1, 4)
# Загрузка облака точек из файла .bin
velo_points = np.fromfile(clustered_ply_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

# Создание облака точек
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points_np)

# Визуализация облака точек
o3d.visualization.draw_geometries([point_cloud])


# Путь к файлу с облаком точек
clustered_ply_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000008.bin'

# Загрузка облака точек из файла .bin
velo_points = np.fromfile(clustered_ply_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]

# Проверка наличия данных в облаке точек
if len(points_np) == 0:
    print("Облако точек пустое или файл не содержит данных.")
else:
    # Масштабирование данных
    scaler = StandardScaler()
    scaled_points_np = scaler.fit_transform(points_np)

    # Отсечение точек снизу по оси Z
    cutting_height = -1.2
    cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
    cut_points_np = points_np[cut_indices]
    cut_scaled_points_np = scaled_points_np[cut_indices]

    # Определение числа ближайших соседей
    n_neighbors = 1

    # Поиск ближайших соседей для каждой точки
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
    distances, indices = nbrs.kneighbors(cut_scaled_points_np)

    # Вычисление среднего расстояния до n_neighbors точек для каждой точки
    avg_distances = np.mean(distances, axis=1)

    # Определение порогового значения для удаления редких точек
    threshold_low = np.percentile(avg_distances, 1)

    # Отфильтровать точки по пороговому значению
    filtered_indices = np.where(avg_distances <= threshold_low)[0]
    filtered_cut_points_np = cut_points_np[filtered_indices]

    # Выполнение кластеризации K-means
    num_clusters = 150
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(filtered_cut_points_np)
    labels = kmeans.labels_
    
    
    
    # Удаление кластеров с количеством точек менее или равным 200
    # Печать кластеров с количеством точек до 200 в каждом кластере
    print("Кластеры с количеством точек до 200 в каждом кластере:")
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_clusters = sorted(zip(unique_labels, counts), key=lambda x: x[1], reverse=True)
    for cluster_label, point_count in sorted_clusters:
        if point_count <= 200:
            print(f"Кластер {cluster_label}: {point_count} точек")
        
    # Удаление кластеров с количеством точек менее или равным 200
    filtered_indices = np.ones(len(labels), dtype=bool)
    for cluster_label, point_count in sorted_clusters:
        if point_count <= 200:
            filtered_indices[labels == cluster_label] = False

    # Фильтрация точек с использованием индексов
    filtered_points_np = filtered_cut_points_np[filtered_indices]

    # Создание облака точек после удаления кластеров
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)

    # Визуализация облака точек после удаления кластеров
    o3d.visualization.draw_geometries([filtered_pcd])



        
    # Присвоение цветов кластерам
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

    # Создание облака точек с цветами кластеров
    kmeans_pcd = o3d.geometry.PointCloud()
    kmeans_pcd.points = o3d.utility.Vector3dVector(filtered_cut_points_np)
    kmeans_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Создание ограничивающего параллелепипеда для каждого кластера
    bbox_list = []
    for label in np.unique(labels):
        if label == -1:
            continue
        cluster_points = filtered_cut_points_np[labels == label]
        bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
        bbox.color = (0, 0, 0)
        bbox_list.append(bbox)

    # Визуализация облака точек с кластеризацией K-means и ограничивающих параллелепипедов
    o3d.visualization.draw_geometries([kmeans_pcd] + bbox_list)

    # Печать кластеров с количеством точек до 200 в каждом кластере
    print("Кластеры с количеством точек до 200 в каждом кластере:")
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_clusters = sorted(zip(unique_labels, counts), key=lambda x: x[1], reverse=True)
    for cluster_label, point_count in sorted_clusters:
        if point_count <= 200:
            print(f"Кластер {cluster_label}: {point_count} точек")
    
    # Удаление кластеров с количеством точек менее или равным 200
filtered_indices = np.ones(len(labels), dtype=bool)
for cluster_label, point_count in sorted_clusters:
    if point_count <= 200:
        filtered_indices[labels == cluster_label] = False

# Фильтрация точек с использованием индексов
filtered_points_np = filtered_cut_points_np[filtered_indices]

# Создание облака точек после удаления кластеров
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)

# Визуализация облака точек после удаления кластеров
o3d.visualization.draw_geometries([filtered_pcd])


from sklearn.cluster import DBSCAN

# Задаем параметры DBSCAN
eps = 0.5  # Радиус окрестности для поиска соседей
min_samples = 10  # Минимальное количество точек в окрестности для образования кластера

# Выполнение кластеризации DBSCAN на облаке точек после удаления кластеров
dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(filtered_points_np)
labels_dbscan = dbscan.labels_

# Присвоение цветов кластерам
max_label_dbscan = labels_dbscan.max()
colors_dbscan = plt.get_cmap("tab20")(labels_dbscan / (max_label_dbscan if max_label_dbscan > 0 else 1))

# Создание облака точек с цветами кластеров после удаления кластеров
dbscan_pcd = o3d.geometry.PointCloud()
dbscan_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)
dbscan_pcd.colors = o3d.utility.Vector3dVector(colors_dbscan[:, :3])

# Создание ограничивающего параллелепипеда для каждого кластера
bbox_list_dbscan = []
for label in np.unique(labels_dbscan):
    if label == -1:
        continue
    cluster_points_dbscan = filtered_points_np[labels_dbscan == label]
    bbox_dbscan = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points_dbscan))
    bbox_dbscan.color = (0, 0, 0)
    bbox_list_dbscan.append(bbox_dbscan)

# Визуализация облака точек с кластеризацией DBSCAN и ограничивающих параллелепипедов после удаления кластеров
o3d.visualization.draw_geometries([dbscan_pcd] + bbox_list_dbscan)

# Печать кластеров с количеством точек в каждом кластере
print("Кластеры с количеством точек:")
unique_labels_dbscan, counts_dbscan = np.unique(labels_dbscan, return_counts=True)
sorted_clusters_dbscan = sorted(zip(unique_labels_dbscan, counts_dbscan), key=lambda x: x[1], reverse=True)
for cluster_label_dbscan, point_count_dbscan in sorted_clusters_dbscan:
    print(f"Кластер {cluster_label_dbscan}: {point_count_dbscan} точек")


# Создание облака точек из массива velo_points
velo_pcd = o3d.geometry.PointCloud()
velo_pcd.points = o3d.utility.Vector3dVector(velo_points[:, :3])

# Визуализация облака точек с кластеризацией DBSCAN и ограничивающих параллелепипедов
o3d.visualization.draw_geometries([velo_pcd] + bbox_list_dbscan)



Кластеры с количеством точек до 200 в каждом кластере:
Кластер 62: 199 точек
Кластер 119: 191 точек
Кластер 82: 188 точек
Кластер 34: 180 точек
Кластер 53: 174 точек
Кластер 131: 173 точек
Кластер 70: 165 точек
Кластер 76: 164 точек
Кластер 109: 164 точек
Кластер 122: 157 точек
Кластер 1: 155 точек
Кластер 133: 155 точек
Кластер 24: 152 точек
Кластер 143: 148 точек
Кластер 89: 140 точек
Кластер 25: 139 точек
Кластер 30: 136 точек
Кластер 117: 132 точек
Кластер 44: 130 точек
Кластер 93: 127 точек
Кластер 144: 124 точек
Кластер 22: 123 точек
Кластер 58: 119 точек
Кластер 71: 115 точек
Кластер 5: 110 точек
Кластер 13: 98 точек
Кластер 35: 98 точек
Кластер 21: 90 точек
Кластер 90: 87 точек
Кластер 96: 85 точек
Кластер 129: 85 точек
Кластер 98: 83 точек
Кластер 139: 81 точек
Кластер 80: 75 точек
Кластер 145: 74 точек
Кластер 134: 72 точек
Кластер 128: 69 точек
Кластер 108: 63 точек
Кластер 59: 60 точек
Кластер 135: 60 точек
Кластер 11: 59 точек
Кластер 18: 58 точек
Кластер 40: 56 точек
Клас

In [13]:
for i, bbox in enumerate(bbox_list_dbscan, 1):
    print(f"Ограничивающий параллелепипед {i}:")
    print(f"Центр: {bbox.get_center()}")
    print(f"Размеры: {bbox.get_extent()}")


Ограничивающий параллелепипед 1:
Центр: [21.61850071  0.39949998  0.15349999]
Размеры: [2.07299995 2.491      1.60299999]
Ограничивающий параллелепипед 2:
Центр: [17.96500015  2.61249989  0.23949999]
Размеры: [1.87599945 2.92499983 1.21899998]
Ограничивающий параллелепипед 3:
Центр: [15.19449997  3.23849988  0.22699998]
Размеры: [0.88899994 1.829      1.04999998]
Ограничивающий параллелепипед 4:
Центр: [18.86900043  5.12699986  0.07950002]
Размеры: [4.26799965 1.64599991 1.699     ]
Ограничивающий параллелепипед 5:
Центр: [ 6.9549998   8.08850014 -0.25850001]
Размеры: [17.00799966 10.74900031  1.88099998]
Ограничивающий параллелепипед 6:
Центр: [ 2.21549994 17.62450027  0.09200001]
Размеры: [1.76299989 1.67499924 1.48800004]
Ограничивающий параллелепипед 7:
Центр: [ 0.663      18.70300007  0.40700001]
Размеры: [1.016      1.8939991  0.93400001]
Ограничивающий параллелепипед 8:
Центр: [-2.98400009 14.97599983 -0.19599998]
Размеры: [3.02800012 3.43199921 1.93199992]
Ограничивающий паралл

In [14]:
filtered_indices = np.ones(len(labels), dtype=bool)
for cluster_label, point_count in sorted_clusters:
    if point_count <= 200:
        filtered_indices[labels == cluster_label] = False

# Фильтрация точек с использованием индексов
filtered_points_np = filtered_cut_points_np[filtered_indices]

# Создание облака точек после удаления кластеров
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points_np)

# Визуализация облака точек после удаления кластеров
o3d.visualization.draw_geometries([filtered_pcd])

# Теперь можно проверить размеры переменных
print("Размеры filtered_points_np:", filtered_points_np.shape)
print("Размеры labels:", labels.shape)


Размеры filtered_points_np: (52872, 3)
Размеры labels: (58481,)


In [15]:
# Фильтрация меток кластеров
filtered_labels = labels[filtered_indices]

# Проверка размерностей
print("Размеры filtered_points_np после фильтрации:", filtered_points_np.shape)
print("Размеры filtered_labels после фильтрации:", filtered_labels.shape)


Размеры filtered_points_np после фильтрации: (52872, 3)
Размеры filtered_labels после фильтрации: (52872,)


In [16]:
import numpy as np

# Путь к файлу с метками объектов
label_file_path = 'D:/Cluster_proj/VIPCClustering/data/KITTI/label_2/000010.txt'

# Чтение меток объектов из файла
with open(label_file_path, 'r') as file:
    lines = file.readlines()

# Создание массива ground_truth_labels
ground_truth_labels = []

# Проход по каждой строке в файле
for line in lines:
    # Разделение строки на отдельные элементы
    elements = line.split()
    # Извлечение типа объекта (метки кластера)
    label = elements[0]
    # Добавление метки кластера в массив
    ground_truth_labels.append(label)

# Преобразование списка в массив numpy
ground_truth_labels = np.array(ground_truth_labels)

# Вывод полученного массива меток кластеров
print(ground_truth_labels)


['Car' 'Car' 'Pedestrian' 'Car' 'Car' 'Car' 'Car' 'Car' 'Car' 'DontCare'
 'DontCare' 'DontCare' 'DontCare']


In [24]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.cluster import silhouette_samples
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial import ConvexHull

def cluster_index(cluster_points, cluster_bb_points):
    return len(cluster_bb_points) / len(cluster_points)

def box_index(cluster_bb_points, total_bb_points):
    return len(cluster_bb_points) / len(total_bb_points)

def label_index(labels, labels_gt):
    return len(np.where(labels == labels_gt)[0]) / len(labels_gt)

def dunn_index(points, labels):
    unique_labels = np.unique(labels)
    min_inter_cluster_distances = np.zeros(len(unique_labels))
    max_intra_cluster_distances = np.zeros(len(unique_labels))
    for i, label in enumerate(unique_labels):
        cluster_points = points[labels == label]
        cluster_distances = pdist(cluster_points)
        min_inter_cluster_distances[i] = np.min(cluster_distances)
        max_intra_cluster_distances[i] = np.max(pairwise_distances(cluster_points))
    return np.min(min_inter_cluster_distances) / np.max(max_intra_cluster_distances)

def dbcv_index(points, labels):
    unique_labels = np.unique(labels)
    k = len(unique_labels)
    intra_cluster_distances = {label: [] for label in unique_labels}
    
    for label in unique_labels:
        cluster_points = points[labels == label]
        cluster_distances = pairwise_distances(cluster_points)
        intra_cluster_distances[label] = np.mean(cluster_distances)
    
    return np.mean(list(intra_cluster_distances.values())) / np.min(squareform(pairwise_distances(points)))



def c_index(points, labels):
    num_points = len(points)
    ssw = 0
    ssb = 0
    overall_centroid = np.mean(points, axis=0)
    for label in np.unique(labels):
        cluster_points = points[labels == label]
        cluster_centroid = np.mean(cluster_points, axis=0)
        ssw += np.sum(np.linalg.norm(cluster_points - cluster_centroid, axis=1) ** 2)
        ssb += len(cluster_points) * np.linalg.norm(cluster_centroid - overall_centroid) ** 2
    return (ssw / (num_points - len(np.unique(labels)))) / (ssb / (len(np.unique(labels)) - 1))

def calinski_harabasz_index(points, labels):
    num_points = len(points)
    overall_centroid = np.mean(points, axis=0)
    ssw = 0
    ssb = 0
    for label in np.unique(labels):
        cluster_points = points[labels == label]
        cluster_centroid = np.mean(cluster_points, axis=0)
        ssw += np.sum(np.linalg.norm(cluster_points - cluster_centroid, axis=1) ** 2)
        ssb += len(cluster_points) * np.linalg.norm(cluster_centroid - overall_centroid) ** 2
    return (ssb / (len(np.unique(labels)) - 1)) / (ssw / (num_points - len(np.unique(labels))))

def composite_external_validation_index(ieb, iec, iel):
    return iel / (iec + ieb) * 2

# Silhouette Score
silhouette_score_custom = silhouette_score(filtered_points_np, filtered_labels)

# Davies-Bouldin Index
davies_bouldin_score_custom = davies_bouldin_score(filtered_points_np, filtered_labels)

# Dunn Index
dunn_index_custom = dunn_index(filtered_points_np, filtered_labels)

# DBCV Index
dbcv_index_custom = dbcv_index(filtered_points_np, filtered_labels)

# C-Index
c_index_custom = c_index(filtered_points_np, filtered_labels)

# Calinski-Harabasz Index
calinski_harabasz_index_custom = calinski_harabasz_index(filtered_points_np, filtered_labels)

# Basic External Validation Indexes
# cluster_indices = []
# box_indices = []
# label_indices = []

# for label in np.unique(filtered_labels):
#     cluster_points = filtered_points_np[filtered_labels == label]
#     cluster_bb_points = cluster_points[np.where(cluster_points[:, 0] != 0)]  # assuming non-zero points correspond to bounding box points
#     total_bb_points = filtered_points_np[np.where(filtered_points_np[:, 0] != 0)]
    
#     cluster_indices.append(cluster_index(cluster_points, cluster_bb_points))
#     box_indices.append(box_index(cluster_bb_points, total_bb_points))
#     label_indices.append(label_index(filtered_labels, ground_truth_labels))  # provide ground truth labels

# mean_cluster_index = np.mean(cluster_indices)
# mean_box_index = np.mean(box_indices)
# mean_label_index = np.mean(label_indices)

# # Composite External Validation Index
# composite_external_validation_index_custom = composite_external_validation_index(mean_box_index, mean_cluster_index, mean_label_index)

print("Silhouette Score:", silhouette_score_custom)
print("Davies-Bouldin Index:", davies_bouldin_score_custom)
print("Dunn Index:", dunn_index_custom)
print("DBCV Index:", dbcv_index_custom)
print("C-Index:", c_index_custom)
print("Calinski-Harabasz Index:", calinski_harabasz_index_custom)
# print("Cluster Index:", mean_cluster_index)
# print("Box Index:", mean_box_index)
# print("Label Index:", mean_label_index)
# print("Composite External Validation Index:", composite_external_validation_index_custom)




ValueError: Distance matrix 'X' must be symmetric.